# Chapter 14. Inheritance: For Better or For Worse

## The Super() Function

consistent use of the `super()` built-in function is essential for maintainable object-oriented Python programs.

In [2]:
from collections import OrderedDict

In [3]:
class LastUpdatedOrderedDict(OrderedDict):

  def __setitem__(self, key, value):
    super().__setitem__(key, value)
    self.move_to_end(key)

In [4]:
a = LastUpdatedOrderedDict()

In [5]:
a['a'] = 1

In [6]:
a['b'] = 2

In [7]:
a

LastUpdatedOrderedDict([('a', 1), ('b', 2)])

In [8]:
a['a'] = 3

In [9]:
a

LastUpdatedOrderedDict([('b', 2), ('a', 3)])

super() call returns a dynamic proxy object that finds a method (such as `__setitem__` in the example) in a superclass of the `type` parameter.

## Subclassing Built-In Types is Tricky

The code of the built-ins (written in C) usually does not call methods overridden by user-defined classes.

In [10]:
class DoppelDict(dict):
  def __setitem__(self, key, value):
    super().__setitem__(key, [value] * 2)

In [16]:
# the __init__ emthod inherited from dict
# clearly ignored that __setitem__ was overridden
dd = DoppelDict(one=1)

In [12]:
dd

{'one': 1}

In [13]:
# [] operator calls our __setitem__
dd['two'] = 2

In [14]:
dd

{'one': 1, 'two': [2, 2]}

In [15]:
# update method from dict does not use our version
# of __setitem__ either
dd.update(three=3)

Late Binding:

> Late Binding: In any call of the form `x.method()`, the exact method to be called must be determined at runtime, based on the class of the receiver `x`.



In [17]:
class AnswerDict(dict):
  def __getitem__(self, key):
    return 42

In [18]:
ad = AnswerDict(a='foo')

In [19]:
ad['a']

42

In [20]:
d = {}
# dict.update method ignored our AnswerDict.__getitem__
d.update(ad)

In [21]:
d['a']

'foo'

In [22]:
d

{'a': 'foo'}

Warning: Instead of subclassing the built-ins, derive your classes from the `collections` module using `UserDict`, `UserList`, and `UserString`

In [26]:
import collections

class DoppelDict2(collections.UserDict):
  def __setitem__(self, key, value):
    super().__setitem__(key, [value] * 2)

In [29]:
dd = DoppelDict2(one=1)

In [30]:
dd

{'one': [1, 1]}

In [31]:
dd['two'] = 2

In [32]:
dd

{'one': [1, 1], 'two': [2, 2]}

In [34]:
dd.update(three=3)

In [35]:
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}